In [1]:
import os
import sys
from pathlib import Path
django_project_dir = Path(".")
sys.path.insert(0, str(django_project_dir))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "newsilkroad.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
import django
django.setup()

[raven.contrib.django.client.DjangoClient]DEBUG 15:23:32 Configuring Raven for host: None
[raven.contrib.django.client.DjangoClient]INFO 15:23:32 Raven is not configured (logging is disabled). Please see the documentation for more information.


In [38]:
import pandas as pd
import geopandas
import matplotlib.pyplot as plt
from infrastructure.models import Project
from locations.models import GeometryStore
import ipdb
from django.db import connection
from shapely.geometry import Point, LineString, Polygon

### Get Geodata objects from the GeometryStore via Projects

In [17]:
sql_lines = Project.objects.filter(geo__isnull=False).filter(geo__lines__geom__isnull=False).values("identifier", "geo__lines__attributes", "geo__lines__geom")
sql_points = Project.objects.filter(geo__isnull=False).filter(geo__points__geom__isnull=False).values("identifier", "geo__points__attributes", "geo__points__geom")

df_lines = geopandas.read_postgis(str(sql_lines.query), connection)
df_points = geopandas.read_postgis(str(sql_points.query), connection)

### Cast UUIDs to String before exporting to GeoJSON

In [18]:
df_lines['identifier'] = df_lines['identifier'].apply(str)
df_lines.to_file("geojson/lines/lines.geojson", driver="GeoJSON")

In [19]:
df_points['identifier'] = df_points['identifier'].apply(str)
df_points.to_file("geojson/points/points.geojson", driver="GeoJSON")

### Get the  Project View table

In [164]:
project_view = pd.read_sql("SELECT * FROM infrastructure_projects_export_view;", connection)
project_view['identifier'] = project_view['identifier'].apply(str)
project_view.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14175 entries, 0 to 14174
Data columns (total 61 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   identifier                     14175 non-null  object 
 1   name                           14175 non-null  object 
 2   infrastructure_type            14169 non-null  object 
 3   countries                      14172 non-null  object 
 4   regions                        14120 non-null  object 
 5   contractors                    3959 non-null   object 
 6   initiatives                    1156 non-null   object 
 7   operators                      754 non-null    object 
 8   funding_sources                1547 non-null   object 
 9   funding_amounts                1547 non-null   object 
 10  funding_currencies             1547 non-null   object 
 11  fuel_type                      10970 non-null  object 
 12  fuel_category                  10970 non-null 

In [140]:
gs = Project.objects.filter(geo__isnull=False).filter(geo__centroid__isnull=False).values("identifier", "geo__centroid")
centroids = pd.DataFrame(gs)
centroids['centroid'] = centroids['geo__centroid'].apply(Point)
centroids['identifier'] = centroids['identifier'].apply(str)
centroids = centroids.drop(columns=["geo__centroid"])

### Merge the centroids into the project view data

In [156]:
replaced = project_view.set_index('identifier').join(centroids.set_index('identifier'))

```NOTE: Turning the identifier into an index removes it from the info() list on the replaced datafram. Giving you 61 columns before and after the join```

In [172]:
geo_replaced = geopandas.GeoDataFrame(replaced, geometry='centroid')
geo_replaced.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 14175 entries, 1de2c76a-9775-43a4-9d9f-c7cf61538d17 to 2e77d1d3-2dcf-4e3c-b256-81bb27f3268e
Data columns (total 61 columns):
 #   Column                         Non-Null Count  Dtype   
---  ------                         --------------  -----   
 0   name                           14175 non-null  object  
 1   infrastructure_type            14169 non-null  object  
 2   countries                      14172 non-null  object  
 3   regions                        14120 non-null  object  
 4   contractors                    3959 non-null   object  
 5   initiatives                    1156 non-null   object  
 6   operators                      754 non-null    object  
 7   funding_sources                1547 non-null   object  
 8   funding_amounts                1547 non-null   object  
 9   funding_currencies             1547 non-null   object  
 10  fuel_type                      10970 non-null  object  
 11  fuel_category           

In [173]:
geo_replaced.columns.difference(project_view.columns)

Index(['centroid'], dtype='object')

In [174]:
project_view.columns.difference(geo_replaced.columns)

Index(['identifier'], dtype='object')

In [175]:
geo_replaced

,name,infrastructure_type,countries,regions,contractors,initiatives,operators,funding_sources,funding_amounts,funding_currencies,...,pipeline_metered,pipeline_throughput,pipeline_throughput_unit,pipeline_throughput_timeframe,pipeline_throughput_year,design_voltage,direct_current,electricity_flow,estimated_transfer_capacity,centroid
identifier,,,,,,,,,,,,,,,,,,,,,
1de2c76a-9775-43a4-9d9f-c7cf61538d17,North-South Road Transport Corridor (Phase 4),Road,'Armenia',"'Russia, Central Asia and the South Caucasus'",None,'International North-South Transport Corridor',None,'Eurasian Development Bank' 'European Investme...,150000000 NULL,USD USD,...,None,NaN,,,None,NaN,None,None,NaN,None
9d46759b-4222-42f6-a5b7-b0c4430f38e7,M6 Vanadzor-Alaverdi-Georgia Interstate Road R...,Road,'Armenia',"'Russia, Central Asia and the South Caucasus'",None,None,None,'Asian Development Bank (ADB)' 'Armenia' 'Euro...,225000 50000000 51000000,USD USD EUR,...,None,NaN,,,None,NaN,None,None,NaN,None
a0050ef7-7d37-4a41-b4e8-9d88a27f1d7d,East-West Highway Improvement,Road,'Azerbaijan',"'Russia, Central Asia and the South Caucasus'","'Impresa S.p.A.', 'Polat Yol Yapı San. ve Tic....",'Transport Corridor Europe-Caucasus-Asia (TRAC...,'Azeravtoyol OJSC','Azerbaijan' 'Asian Development Bank (ADB)' 'S...,73800000 40400000 13000000 10400000,USD USD USD USD,...,None,NaN,,,None,NaN,None,None,NaN,POINT (46.22690 40.86572)
729efb76-3a6c-4879-871e-45b9685821df,Highway 2,Road,'Azerbaijan',"'Russia, Central Asia and the South Caucasus'","'Makyol İnşaat A.Ş.', 'KOLIN Construction, Tou...",'Azerbaijan State Programme on Poverty Reducti...,None,'Azerbaijan' 'International Bank for Reconstru...,64870000 200000000,USD USD,...,None,NaN,,,None,NaN,None,None,NaN,POINT (48.87727 40.39947)
efca2104-5465-4e13-a869-e23d96faf30b,Baku-Tbilisi-Kars Railway,Rail,"'Azerbaijan', 'Georgia', 'Turkey'","'Russia, Central Asia and the South Caucasus'",'Özgün Yapı & Çelikler Consortium (Kars-Akhalk...,"'Lapis Lazuli Route', 'Silk Wind', 'Azerbaijan...",None,'Azerbaijan' 'Georgia',775000000 79440000,USD USD,...,None,NaN,,,None,NaN,None,None,NaN,POINT (46.21902 40.85021)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
e396bbec-564f-4b08-a0d2-b967a8a8e63d,Vukosavlje-Doboj Highway Project (Vukosavlje-B...,Road,'Bosnia and Herzegovina','Eastern Europe and Balkans',None,"'CEEC ""16+1”', 'Belt and Road'",None,None,None,None,...,None,NaN,,,None,NaN,None,None,NaN,POINT (18.04077 45.36766)
c1f67b80-deb2-4cb6-8daf-295fa4784638,"Modernization of Rijeka-Zagreb Rail, Section K...",Rail,'Croatia','Eastern Europe and Balkans','China Road and Bridge Corporation',None,None,None,None,None,...,None,NaN,,,None,NaN,None,None,NaN,POINT (15.37388 45.32569)
aeaece46-19b0-4588-8562-f6793524ff12,"Sumsel-5 Coal-fired Power Plant, South Sumatra",None,None,None,None,None,None,None,None,None,...,None,NaN,,,None,NaN,None,None,NaN,POINT (103.75455 -2.15618)


In [176]:
geo_replaced.to_file("geojson/projects/projects.geojson", driver="GeoJSON")